In [1]:
import findspark
findspark.init()
import pandas as pd
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit, when, concat_ws, upper, lower, avg, sum, count, udf, max, min,concat_ws, upper, lower, length, substring, instr, trim, lpad, rpad
from pyspark.sql.types import StringType, IntegerType, DoubleType


In [2]:
spark = SparkSession.builder.appName("AdvancedSQL_UDFs_DepartmentEmployee").getOrCreate()

In [3]:
data_dept = [
    (1, "logistics"),
    (2, "Accounts"),
    (3, "DS"),
    (4, "Marketing")
]

columns_dept = ["dept_id", "dept_name"]


In [4]:
data_emp = [
    (1, "A", 34, 1, 50000),
    (2, "B", 45, 2, 60000),
    (3, "C", 29, 3, 55000),
    (4, "D", 40, 1, 70000),
    (5, "E", 31, 2, 48000),
    (6, "F", 38, 3, 72000)
]

columns_emp = ["emp_id", "name", "age", "dept_id", "salary"]


In [5]:
# Create DataFrames
df_emp = spark.createDataFrame(data_emp, columns_emp)
df_dept = spark.createDataFrame(data_dept, columns_dept)

# Display sample data
df_emp.show()
df_dept.show()


+------+----+---+-------+------+
|emp_id|name|age|dept_id|salary|
+------+----+---+-------+------+
|     1|   A| 34|      1| 50000|
|     2|   B| 45|      2| 60000|
|     3|   C| 29|      3| 55000|
|     4|   D| 40|      1| 70000|
|     5|   E| 31|      2| 48000|
|     6|   F| 38|      3| 72000|
+------+----+---+-------+------+

+-------+---------+
|dept_id|dept_name|
+-------+---------+
|      1|logistics|
|      2| Accounts|
|      3|       DS|
|      4|Marketing|
+-------+---------+



In [6]:
df_emp.show()

+------+----+---+-------+------+
|emp_id|name|age|dept_id|salary|
+------+----+---+-------+------+
|     1|   A| 34|      1| 50000|
|     2|   B| 45|      2| 60000|
|     3|   C| 29|      3| 55000|
|     4|   D| 40|      1| 70000|
|     5|   E| 31|      2| 48000|
|     6|   F| 38|      3| 72000|
+------+----+---+-------+------+



In [7]:
# Aggregations by department
df_emp.groupBy("dept_id").agg(
    avg("salary").alias("avg_salary")).show()


+-------+----------+
|dept_id|avg_salary|
+-------+----------+
|      1|   60000.0|
|      2|   54000.0|
|      3|   63500.0|
+-------+----------+



In [8]:
df_emp.groupBy("dept_id").agg(
    sum("salary").alias("total_salary"),
    max("salary").alias("max_salary"),
    min("salary").alias("min_salary")).show()

+-------+------------+----------+----------+
|dept_id|total_salary|max_salary|min_salary|
+-------+------------+----------+----------+
|      1|      120000|     70000|     50000|
|      2|      108000|     60000|     48000|
|      3|      127000|     72000|     55000|
+-------+------------+----------+----------+



In [9]:
df_emp.groupBy("dept_id").agg(count("emp_id").alias("employee_count")).show()

+-------+--------------+
|dept_id|employee_count|
+-------+--------------+
|      1|             2|
|      2|             2|
|      3|             2|
+-------+--------------+



In [10]:
# String Functions
df_emp.withColumn("name_upper", upper(col("name"))).withColumn("name_lower", lower(col("name"))).show()


+------+----+---+-------+------+----------+----------+
|emp_id|name|age|dept_id|salary|name_upper|name_lower|
+------+----+---+-------+------+----------+----------+
|     1|   A| 34|      1| 50000|         A|         a|
|     2|   B| 45|      2| 60000|         B|         b|
|     3|   C| 29|      3| 55000|         C|         c|
|     4|   D| 40|      1| 70000|         D|         d|
|     5|   E| 31|      2| 48000|         E|         e|
|     6|   F| 38|      3| 72000|         F|         f|
+------+----+---+-------+------+----------+----------+



In [11]:
# 2. Find length of employee names
df_emp.withColumn("name_length", length(col("name"))).show()


+------+----+---+-------+------+-----------+
|emp_id|name|age|dept_id|salary|name_length|
+------+----+---+-------+------+-----------+
|     1|   A| 34|      1| 50000|          1|
|     2|   B| 45|      2| 60000|          1|
|     3|   C| 29|      3| 55000|          1|
|     4|   D| 40|      1| 70000|          1|
|     5|   E| 31|      2| 48000|          1|
|     6|   F| 38|      3| 72000|          1|
+------+----+---+-------+------+-----------+



In [12]:
# 3. Extract substring of name (first 5 characters)
df_emp.withColumn("name_substring", substring(col("name"), 1, 5)).show()

# 4. Find position of a substring (e.g., 'Smith' in name)
df_emp.withColumn("pos_smith", instr(col("name"), "Smith")).show()

# 5. Trim spaces from employee names
df_emp.withColumn("trimmed_name", trim(col("name"))).show()

# 6. Concatenate name and department ID
df_emp.withColumn("emp_info", concat_ws(" - ", col("name"), col("dept_id"))).show()




+------+----+---+-------+------+--------------+
|emp_id|name|age|dept_id|salary|name_substring|
+------+----+---+-------+------+--------------+
|     1|   A| 34|      1| 50000|             A|
|     2|   B| 45|      2| 60000|             B|
|     3|   C| 29|      3| 55000|             C|
|     4|   D| 40|      1| 70000|             D|
|     5|   E| 31|      2| 48000|             E|
|     6|   F| 38|      3| 72000|             F|
+------+----+---+-------+------+--------------+

+------+----+---+-------+------+---------+
|emp_id|name|age|dept_id|salary|pos_smith|
+------+----+---+-------+------+---------+
|     1|   A| 34|      1| 50000|        0|
|     2|   B| 45|      2| 60000|        0|
|     3|   C| 29|      3| 55000|        0|
|     4|   D| 40|      1| 70000|        0|
|     5|   E| 31|      2| 48000|        0|
|     6|   F| 38|      3| 72000|        0|
+------+----+---+-------+------+---------+

+------+----+---+-------+------+------------+
|emp_id|name|age|dept_id|salary|trimmed_na

In [13]:
# Conditional Expressions
df_emp.withColumn("salary_level", when(col("salary") >= 70000, "High")
                                    .when(col("salary") >= 55000, "Medium")
                                    .otherwise("Low")).show()

# Concatenate employee details
df_emp.withColumn("emp_info", concat_ws(" - ", col("name"), col("age"), col("salary"))).show()


+------+----+---+-------+------+------------+
|emp_id|name|age|dept_id|salary|salary_level|
+------+----+---+-------+------+------------+
|     1|   A| 34|      1| 50000|         Low|
|     2|   B| 45|      2| 60000|      Medium|
|     3|   C| 29|      3| 55000|      Medium|
|     4|   D| 40|      1| 70000|        High|
|     5|   E| 31|      2| 48000|         Low|
|     6|   F| 38|      3| 72000|        High|
+------+----+---+-------+------+------------+

+------+----+---+-------+------+--------------+
|emp_id|name|age|dept_id|salary|      emp_info|
+------+----+---+-------+------+--------------+
|     1|   A| 34|      1| 50000|A - 34 - 50000|
|     2|   B| 45|      2| 60000|B - 45 - 60000|
|     3|   C| 29|      3| 55000|C - 29 - 55000|
|     4|   D| 40|      1| 70000|D - 40 - 70000|
|     5|   E| 31|      2| 48000|E - 31 - 48000|
|     6|   F| 38|      3| 72000|F - 38 - 72000|
+------+----+---+-------+------+--------------+



In [14]:
#joins
# Inner Join
df_emp.join(df_dept, df_emp.dept_id == df_dept.dept_id, "inner").show()

# Left Join
df_emp.join(df_dept, df_emp.dept_id == df_dept.dept_id, "left").show()

# Right Join
df_emp.join(df_dept, df_emp.dept_id == df_dept.dept_id, "right").show()

# Full Outer Join
df_emp.join(df_dept, df_emp.dept_id == df_dept.dept_id, "outer").show()

# Cross Join
df_emp.crossJoin(df_dept).show(10)  # limiting to 10 rows for readability



+------+----+---+-------+------+-------+---------+
|emp_id|name|age|dept_id|salary|dept_id|dept_name|
+------+----+---+-------+------+-------+---------+
|     1|   A| 34|      1| 50000|      1|logistics|
|     4|   D| 40|      1| 70000|      1|logistics|
|     2|   B| 45|      2| 60000|      2| Accounts|
|     5|   E| 31|      2| 48000|      2| Accounts|
|     3|   C| 29|      3| 55000|      3|       DS|
|     6|   F| 38|      3| 72000|      3|       DS|
+------+----+---+-------+------+-------+---------+

+------+----+---+-------+------+-------+---------+
|emp_id|name|age|dept_id|salary|dept_id|dept_name|
+------+----+---+-------+------+-------+---------+
|     1|   A| 34|      1| 50000|      1|logistics|
|     2|   B| 45|      2| 60000|      2| Accounts|
|     3|   C| 29|      3| 55000|      3|       DS|
|     4|   D| 40|      1| 70000|      1|logistics|
|     5|   E| 31|      2| 48000|      2| Accounts|
|     6|   F| 38|      3| 72000|      3|       DS|
+------+----+---+-------+-----

In [15]:
# UDF  Categorize employees by age
def categorize_age(age):
    if age < 30:
        return "Young"
    elif age < 40:
        return "Mid-age"
    else:
        return "Senior"

age_category_udf = udf(categorize_age, StringType())
df_emp.withColumn("age_category", age_category_udf(col("age"))).show()


+------+----+---+-------+------+------------+
|emp_id|name|age|dept_id|salary|age_category|
+------+----+---+-------+------+------------+
|     1|   A| 34|      1| 50000|     Mid-age|
|     2|   B| 45|      2| 60000|      Senior|
|     3|   C| 29|      3| 55000|       Young|
|     4|   D| 40|      1| 70000|      Senior|
|     5|   E| 31|      2| 48000|     Mid-age|
|     6|   F| 38|      3| 72000|     Mid-age|
+------+----+---+-------+------+------------+



In [16]:
# UDF  Categorize employees by age
def categorize_salary(salary):
    if salary < 30000:
        return "Class C"
    elif salary > 50000:
        return "Class B"
    else:
        return "Class A"

salary_category_udf = udf(categorize_salary, StringType())
df_emp.withColumn("salary_category", salary_category_udf(col("salary"))).show()


+------+----+---+-------+------+---------------+
|emp_id|name|age|dept_id|salary|salary_category|
+------+----+---+-------+------+---------------+
|     1|   A| 34|      1| 50000|        Class A|
|     2|   B| 45|      2| 60000|        Class B|
|     3|   C| 29|      3| 55000|        Class B|
|     4|   D| 40|      1| 70000|        Class B|
|     5|   E| 31|      2| 48000|        Class A|
|     6|   F| 38|      3| 72000|        Class B|
+------+----+---+-------+------+---------------+

